In [ ]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *
from sbl_bbl import *

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_ct2_v2'
turbmethods = ['SMCLT-H15', 'SMCLT-KC04', 'KPPLT-LF17']

datapath = os.path.join(os.path.pardir, 'tests', '{:s}'.format(casename))
filepath = os.path.join(datapath, 'averages.jld2')
data_pfl = OceananigansDataProfile(filepath=filepath)
ds_ocgn = data_pfl.dataset
ds_gotm = {}
gotm_dir = os.path.join(os.path.pardir, 'gotm', 'run', '{:s}_l144'.format(casename))
for turbmethod in turbmethods:
    gotm_sim = Simulation(path=os.path.join(gotm_dir, '{:s}-Rlx'.format(turbmethod)))
    ds_gotm[turbmethod] = gotm_sim.load_data()
gotm_sim = Simulation(path=os.path.join(gotm_dir, 'SMCLT-H15'))
ds_gotm['SMCLT-H15-0'] = gotm_sim.load_data()

In [ ]:
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
g = 9.81
H = 30
N2 = 1.962e-4
bstar = N2 * H
u10 = 8
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
cp = 3991
alphaT = 2.0e-4
lat = 45
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
Q0 = 150
B0 = Q0*alphaT*g/rhoo/cp
wstar = (B0*H)**(1/3)
Ti = inertial_period(lat)
print('Ti = {:6.3f}'.format(Ti))
z1 = ds_ocgn.z[0].data+H
z0 = 0.01
kappa = 0.4
tslice = slice('2000-01-11 00:00:00', '2000-01-13 00:00:00')
ub = ds_ocgn.data_vars['u'].isel(z=0).sel(time=tslice).mean(dim='time').data
vb = ds_ocgn.data_vars['v'].isel(z=0).sel(time=tslice).mean(dim='time').data
cdb = kappa**2/np.log(z1/z0)**2
ustarb = kappa/np.log(z1/z0)*np.sqrt(ub**2+vb**2)
print('u*^b = {:6.3f}'.format(ustarb))

In [ ]:
print('---- Oceananigans ----')
N2_ocgn = nondim_da(ds_ocgn.data_vars['b'].differentiate(coord='z')/N2, H=H, Tf=Ti)
tslice1 = get_tslice(N2_ocgn, Ti)
edges1_les, edges2_les = get_edges(N2_ocgn)
print('---- GOTM No Relex ----')
N2_gotm = nondim_da(ds_gotm['SMCLT-H15-0'].data_vars['temp'].differentiate(coord='z')[:,:,0,0]*alphaT*g/N2, H=H, Tf=Ti)
tslice2 = get_tslice(N2_gotm, Ti)

In [ ]:
def get_das_ocgn(ds):
    das_ocgn = dict(
        wb = (ds.data_vars['wb']+ds.data_vars['wbsb'])/B0,
        wu = (ds.data_vars['wu']+ds.data_vars['wusb'])/ustarb**2,
        wv = (ds.data_vars['wv']+ds.data_vars['wvsb'])/ustarb**2,
        )
    return das_ocgn

def get_das_gotm(ds):
    das_gotm = dict(
        wb = get_flux(ds.data_vars['temp'].squeeze(),
                      ds.data_vars['nuh'].squeeze(),
                      ds.data_vars['gamh'].squeeze())*alphaT*g/B0,
        wu = (get_flux(ds.data_vars['u'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamu'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dusdz'].squeeze())/ustarb**2,
        wv = (get_flux(ds.data_vars['v'].squeeze(),
                      ds.data_vars['num'].squeeze(),
                      ds.data_vars['gamv'].squeeze())
              -ds.data_vars['nucl'].squeeze()*ds.data_vars['dvsdz'].squeeze())/ustarb**2,
    )
    return das_gotm

In [ ]:
labels = dict(
    wb = '$\overline{w^\prime b^\prime}/B_0$',
    wu = '$\overline{w^\prime u^\prime}/u_{*b}^2$',
    wv = '$\overline{w^\prime v^\prime}/u_{*b}^2$',
)
levels = dict(
    wb = np.linspace(-4, 4, 41),
    wu = np.linspace(-2, 2, 41),
    wv = np.linspace(-1.2, 1.2, 41),
)
turbmethods = ['SMCLT-H15', 'KPPLT-LF17']
rlcolor = {'RdBu_r': 'k', 'viridis': 'w', 'bone': 'w', 'bone_r': 'k', 'pink_r': 'k'}
cmap='RdBu_r'
abc = dict(
    wb = 'ad',
    wu = 'be',
    wv = 'cf',
)

fig, axarr = plt.subplots(3, 2, sharex='col')
fig.set_size_inches([10, 5])
rkey = 'r1'
cf = {}
tslc = tslice1
for i, turbmethod in enumerate(turbmethods):
    ax = axarr[0,i]
    ax.text(0.98, 0.9, turbmethod[6:], transform=ax.transAxes, fontsize=10, weight='bold', va='top', ha='right')
    ax.text((tslc[0].start+tslc[0].stop)/2, -0.05, 'T1', va='top', ha='center')
    ax.text((tslc[1].start+tslc[1].stop)/2, -0.05, 'T2', va='top', ha='center')
    ds = ds_gotm[turbmethod]
    das_gotm = nondim_das(get_das_gotm(ds), H=H, Tf=Ti)
    for j, var in enumerate(labels.keys()):
        ax = axarr[j,i]
        cf[var] = das_gotm[var].plot(ax=ax, levels=levels[var], cmap=cmap, extend='both', add_colorbar=False)
        for eg in [edges1_les, edges2_les]:
            eg.plot(ax=ax, color='k', linewidth=0.75)
        for xx in [tslc[0].start, tslc[0].stop, tslc[1].start, tslc[1].stop]:
            ax.axvline(x=xx, linestyle=':', color=rlcolor[cmap])
        ax.set_title('')
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.set_xlim([0,16])
        ax.text(0.98, 0.1, '({:s})'.format(abc[var][i]), transform=ax.transAxes, va='bottom', ha='right')

plt.subplots_adjust(top=0.97, bottom=0.1, left=0.08, right=0.87, hspace=0.15, wspace=0.15)

for j in np.arange(2):
    for i, var in enumerate(labels.keys()):
        ax = axarr[i,1]
        pos = ax.get_position()
        cax = plt.axes([0.9, pos.y0, 0.015, pos.height])
        cb = plt.colorbar(cf[var], cax=cax)
        cb.set_label(labels[var])
        ax = axarr[i,0]
        ax.set_ylabel('$z/H$')
    ax = axarr[-1,j]
    ax.set_xlabel('$t/T_f$')
figname = os.path.join(figpath, 'mean_gotm_relax')
fig.savefig(figname, dpi = 300, facecolor='w')

In [ ]:
labels = dict(
    wb = '$\overline{w^\prime b^\prime}/B_0$',
    wu = '$\overline{w^\prime u^\prime}/u_{*b}^2$',
    wv = '$\overline{w^\prime v^\prime}/u_{*b}^2$',
)
turbmethods = ['SMCLT-H15', 'KPPLT-LF17']
colors = ['tab:blue', 'tab:red', 'tab:purple']
linestyles = ['-', '-', '-']
kw_line_les  = dict(linestyle='-', color='k')
kw_line_gotm = {}
for i, turbmethod in enumerate(turbmethods):
    kw_line_gotm[turbmethod] = dict(linestyle=linestyles[i], color=colors[i])
    
abc = dict(
    wb = 'ad',
    wu = 'be',
    wv = 'cf',
)
abc_loc = dict(
    wb = ['left', 'right'],
    wu = ['left', 'left'],
    wv = ['left', 'left'],
)
abc_x = {'left': 0.1, 'right': 0.9}

titles = ['T1', 'T2']

fig, axarr = plt.subplots(2, 3, sharey='row')
fig.set_size_inches([7, 5])
das_ocgn = nondim_das(get_das_ocgn(ds_ocgn), H=H, Tf=Ti)
das_gotm = {}
das_gotm_mflux = {}
for turbmethod in turbmethods:
    ds = ds_gotm[turbmethod]
    das_gotm[turbmethod] = nondim_das(get_das_gotm(ds), H=H, Tf=Ti)
for j in np.arange(2):
    for i, var in enumerate(labels.keys()):
        ax = axarr[j, i]
        da = das_ocgn[var].sel(time=tslice1[j]).mean(dim='time')
        da.plot(ax=ax, y=da.dims[0], **kw_line_les, label='LES')
        for turbmethod in turbmethods:
            da = das_gotm[turbmethod][var].sel(time=tslice1[j]).mean(dim='time')
            da.plot(ax=ax, y=da.dims[0], **kw_line_gotm[turbmethod], label=turbmethod[6:])
        ax.text(abc_x[abc_loc[var][j]], 0.9, '({:s})'.format(abc[var][j]), transform=ax.transAxes, va='top', ha=abc_loc[var][j])
        if i == 0:
            ax.text(0.01, 1.0, titles[j], transform=ax.transAxes, va='bottom', ha='left', fontsize=10)
            if j == 1:
                ax.legend(loc='lower right', fontsize=9)
        ax.set_title('')
        ax.set_ylabel('')
        ax.set_ylim([-1, 0])
        ax.set_xlabel(labels[var])

for i in np.arange(2):
    ax = axarr[i,0]
    ax.set_ylabel('$z/H$')

plt.subplots_adjust(top=0.94, bottom=0.12, left=0.12, right=0.97, hspace=0.35, wspace=0.15)
figname = os.path.join(figpath, 'profiles_relax')
fig.savefig(figname, dpi = 300, facecolor='w')